In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/hong-kong-weather-20082016'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hong-kong-weather-20082016/weather.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Assuming your dataset is in CSV format
df = pd.read_csv('/kaggle/input/hong-kong-weather-20082016/weather.csv')

# Preview the first few rows of the data
df.head()


,mean_temp,Grass_Min_Temp,max_Temp,mean_amount_of_cloud,mean_dew_point_temp,mean_wet_bulb_temp,min_temp,pressure,rainfall,RH,date
0,17.7,12.4,20.7,32.0,13.4,15.2,15.5,1018.7,0,77.0,1970-01-01
1,19.5,12.8,22.9,44.0,14.1,16.4,17.2,1018.6,0,72.0,1970-01-02
2,18.8,14.7,21.2,35.0,13.3,15.7,17.5,1019.2,0,71.0,1970-01-03
3,17.9,13.8,20.7,20.0,13.1,15.2,16.1,1020.6,0,74.0,1970-01-04
4,14.7,13.1,18.2,66.0,7.4,11.1,11.8,1026.0,Trace,62.0,1970-01-05


In [4]:
df = df[df['rainfall'] != "Trace"]
df = df.dropna(how='all')

In [5]:
df = df.reset_index(drop=True)

In [6]:
df.head()

,mean_temp,Grass_Min_Temp,max_Temp,mean_amount_of_cloud,mean_dew_point_temp,mean_wet_bulb_temp,min_temp,pressure,rainfall,RH,date
0,17.7,12.4,20.7,32.0,13.4,15.2,15.5,1018.7,0,77.0,1970-01-01
1,19.5,12.8,22.9,44.0,14.1,16.4,17.2,1018.6,0,72.0,1970-01-02
2,18.8,14.7,21.2,35.0,13.3,15.7,17.5,1019.2,0,71.0,1970-01-03
3,17.9,13.8,20.7,20.0,13.1,15.2,16.1,1020.6,0,74.0,1970-01-04
4,13.8,9.5,16.9,63.0,6.7,10.3,10.4,1025.6,0,63.0,1970-01-06


In [7]:
# Convert 'date' to datetime
df['date'] = pd.to_datetime(df['date'])

# Extract features from the date column
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# Drop the original 'date' column
df = df.drop('date', axis=1)

In [8]:
print(df.dtypes)

mean_temp               float64
Grass_Min_Temp           object
max_Temp                float64
mean_amount_of_cloud    float64
mean_dew_point_temp     float64
mean_wet_bulb_temp      float64
min_temp                float64
pressure                float64
rainfall                 object
RH                      float64
year                      int32
month                     int32
day                       int32
dtype: object


In [9]:
# Convert all columns to integers
for column in df.select_dtypes(include=['object']).columns:
    # Attempt to convert each 'object' column to integers
    df[column] = pd.to_numeric(df[column], errors='coerce').astype('float64')

# Preview the DataFrame
print(df)

       mean_temp  Grass_Min_Temp  max_Temp  mean_amount_of_cloud  \
0           17.7            12.4      20.7                  32.0   
1           19.5            12.8      22.9                  44.0   
2           18.8            14.7      21.2                  35.0   
3           17.9            13.8      20.7                  20.0   
4           13.8             9.5      16.9                  63.0   
...          ...             ...       ...                   ...   
14638       20.1            16.7      22.9                  65.0   
14639       21.0            16.5      24.5                  29.0   
14640       18.0            13.5      21.3                  23.0   
14641       18.8            16.9      19.6                  92.0   
14642       20.3            18.7      22.5                  90.0   

       mean_dew_point_temp  mean_wet_bulb_temp  min_temp  pressure  rainfall  \
0                     13.4                15.2      15.5    1018.7       0.0   
1                     1

In [12]:
# Define the target variable (mean temperature) and features
X = df.drop('mean_temp', axis=1)  # Features
y = df['mean_temp']  # Target variable
# Drop rows with any NaN values in X
X = X.dropna()

# Ensure the target variable y also aligns after dropping rows in X
y = y[X.index]


# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n estimators of 100 which is then regulated if needed

In [13]:
# Initialize the RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf.predict(X_test)

In [14]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R-squared: {r2}")

MAE: 0.1910003431708992
MSE: 0.07070997700754969
R-squared: 0.9973970540162185


In [15]:
y_pred

array([29.404, 27.523, 23.479, ..., 30.239, 15.789, 12.259])

In [17]:
mean_temp = df.mean_temp.mean()
mean_temp

23.29959024790002

In [18]:
error_percent = (mae/mean_temp)*100
error_percent

0.8197583783178932

In [19]:
percentage_accuracy = 100-error_percent
percentage_accuracy

99.1802416216821